In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "monologg/kobert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)


In [2]:
import torch
model = torch.load('model.pt')

In [23]:
import pandas as pd
data = pd.read_csv('text.csv')
data.head()

,Unnamed: 0,1,2,label
0,0,구슬 탈출 2,스타트링크에서 판매하는 어린이용 장난감 중에서 가장 인기가 많은 제품은 구슬 탈출이...,0
1,1,2048 (Easy),2048 게임은 4×4 크기의 보드에서 혼자 즐기는 재미있는 게임이다. 이 링크를 ...,0
2,2,뱀,"'Dummy' 라는 도스게임이 있다. 이 게임에는 뱀이 나와서 기어다니는데, 사과를...",0
3,3,시험 감독,"총 N개의 시험장이 있고, 각각의 시험장마다 응시자들이 있다. i번 시험장에 있는 ...",0
4,4,주사위 굴리기,"크기가 N×M인 지도가 존재한다. 지도의 오른쪽은 동쪽, 위쪽은 북쪽이다. 이 지도...",0


In [24]:
# Train / Test set 분리
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

train shape is: 73
test shape is: 19


In [25]:
texts = list(data['2'])
labels = list(data['label'])


In [26]:
import torch
encoding = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
labels = torch.tensor(labels)


In [27]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)


In [4]:
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            padding='max_length',  # Pad to the maximum length in the batch
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(label),
        }

In [3]:
# Create DataLoader
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
test_dataset = CustomDataset(test_texts, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


NameError: name 'train_texts' is not defined

In [9]:
def evaluate(model, dataloader):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in dataloader:
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=1)
            all_labels.extend(batch["labels"].numpy())
            all_predictions.extend(predictions.numpy())

    return all_labels, all_predictions

In [35]:
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, roc_auc_score

# # Define the classifier on top of KoBERT
# classifier = nn.Linear(model.config.hidden_size, 2)  # Assuming binary classification

# # Combine KoBERT model with classifier
# model.classifier = classifier

# Define optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

# Train the model
for epoch in range(3):  # Adjust the number of epochs as needed
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = criterion(outputs.logits, batch["labels"])
        loss.backward()
        optimizer.step()

    all_labels, all_predictions = evaluate(model, test_loader)

    # Calculate accuracy and ROC-AUC
    accuracy = accuracy_score(all_labels, all_predictions)
    roc_auc = roc_auc_score(all_labels, all_predictions)

    print(f"Epoch {epoch + 1} - Accuracy: {accuracy:.4f}, ROC-AUC: {roc_auc:.4f}")


<ipython-input-30-92c5feefae2e>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(label),


Epoch 1 - Accuracy: 0.7368, ROC-AUC: 0.6429


<ipython-input-30-92c5feefae2e>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(label),


Epoch 2 - Accuracy: 1.0000, ROC-AUC: 1.0000


<ipython-input-30-92c5feefae2e>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(label),


Epoch 3 - Accuracy: 1.0000, ROC-AUC: 1.0000


In [36]:
final_all_labels, final_all_predictions = evaluate(model, test_loader)
final_accuracy = accuracy_score(final_all_labels, final_all_predictions)
final_roc_auc = roc_auc_score(final_all_labels, final_all_predictions)

print(f"Final Accuracy: {final_accuracy:.4f}, Final ROC-AUC: {final_roc_auc:.4f}")

<ipython-input-30-92c5feefae2e>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(label),


Final Accuracy: 1.0000, Final ROC-AUC: 1.0000


In [37]:
torch.save(model.cpu().state_dict(), "model.pt")

In [5]:
import pandas as pd
valid_data = pd.read_csv('validation_kakao.csv', header=None)
valid_data.head()

,0,1,2,3
0,0,상금 헌터,"2017년에 이어, 2018년에도 카카오 코드 페스티벌이 개최된다! 카카오 코드 페...",1
1,1,인형들,카카오프렌즈 스토어에서는 N종류의 인형을 팔고 있다. N개의 인형들 중에서는 잘 팔...,1
2,2,숏코딩,코드 페스티벌 온라인 예선에 참가하고 있던 라이언은 이제 남은 시간이 00:00:0...,1
3,3,부스터,최근 새로 출시되어 인기를 끌고 있는 카카오게임이 있다. 이 게임에서는 2차원 좌표...,1
4,4,음악 추천,멜론은 국내 최다 고객과 음원을 보유한 디지털 뮤직 플랫폼이다. 멜론의 특징 중 하...,1


In [6]:
valid_texts = list(valid_data[2])
valid_labels = list(valid_data[3])

In [7]:
valid_dataset = CustomDataset(valid_texts, valid_labels, tokenizer)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False)

In [10]:
valid_all_labels, valid_all_predictions = evaluate(model, valid_loader)
valid_accuracy = accuracy_score(valid_all_labels, valid_all_predictions)
valid_roc_auc = roc_auc_score(valid_all_labels, valid_all_predictions)

NameError: name 'accuracy_score' is not defined